In [ ]:
# Radar Chart Function for Specific Metrics
def radar_chart_percentiles(df, percentiles, athlete_name, athlete_id, pdf, unit_dict, verbose=False):
    """
    Generate a radar chart comparing an athlete's onboarding and most recent data, ensuring correct data alignment.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        percentiles (dict): Pre-calculated percentiles.
        athlete_name (str): Athlete's full name.
        athlete_id (int): Athlete ID.
        pdf (PdfPages): PDF object for saving the plot.
        unit_dict (dict): Dictionary of units for metrics.
        verbose (bool): Enable verbose logging for debugging.

    Returns:
        None
    """
    # Retrieve the athlete's current level
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    athlete_level = athlete_data['Level'].iloc[-1]

    # Filter data for the athlete's current level
    level_data = athlete_data[athlete_data['Level'] == athlete_level]
    if level_data.empty:
        print(f"No data available for athlete at level '{athlete_level}'.")
        return

    # Define metrics for the radar chart
    selected_metrics = [
        ('Weigh-in', 'Weigh-in', 'Weight'),
        ('Weigh-in', 'Height', 'Height'),
        ('Trackman Bullpen', 'Fastball', 'Max Velo'),
        ('Trackman Bullpen', 'Fastball', 'Average Velo'),
        ('Blast Motion', 'Front Toss', 'Max Bat Speed'),
        ('Blast Motion', 'Front Toss', 'Peak Hand Speed'),
        ('HitTrax', 'Front Toss', 'Average EV'),
        ('HitTrax', 'Front Toss', 'Max EV'),
        ('HitTrax', 'Front Toss', 'Max Distance'),
        ('Roll Ins', '5oz', 'Max Velo'),
        ('Double Plays', '5oz', 'Max Velo'),
        ('Turn and Burns', '5oz', 'Max Velo'),
        ('Pulldowns', '5oz', 'Max Velo'),
        ('Jump', 'Broad', 'Distance'),
        ('Jump', 'Vertical', 'Vert'),
        ('Back Squat', 'Strength-Speed', 'Weight'),
        ('Deadlift', 'Strength-Speed', 'Weight'),
        ('Bench Press', 'Strength-Speed', 'Weight'),
        ('ArmCare', 'Fresh Exam', 'Arm Score'),
    ]

    # Define custom labels for the metrics
    custom_labels = [
        "Body Weight",
        "Height",
        "Fastball Max Velocity",
        "Fastball Average Velocity",
        "Max Bat Speed",
        "Peak Hand Speed",
        "Average Exit Velo",
        "Max Exit Velo",
        "Max Distance",
        "5oz Roll Ins",
        "5oz Double Plays",
        "5oz Turn and Burns",
        "5oz Pulldown",
        "Broad Jump Distance",
        "Vertical Jump Height",
        "Speed Back Squat",
        "Speed Deadlift",
        "Speed Bench Press",
        "ArmCare Score",
    ]

    # Initialize values for plotting
    labels = []
    onboarding_values = []
    recent_values = []
    onboarding_actuals = []
    recent_actuals = []

    for i, (test_type, sub_type, metric) in enumerate(selected_metrics):
        metric_data = level_data[
            (level_data['Test Type'] == test_type) & (level_data['Test Sub-Type'] == sub_type)
        ]
        if metric_data.empty:
            labels.append(f"{custom_labels[i]}\n(N/A)")
            onboarding_values.append(0)
            recent_values.append(0)
            onboarding_actuals.append(None)
            recent_actuals.append(None)
            continue

        # Extract most recent test by max Test Number
        most_recent_test = metric_data.loc[metric_data['Test Number'].idxmax()]
        onboarding_test = metric_data.loc[metric_data['Test Number'].idxmin()]

        # Validate the extracted test
        if verbose:
            print(f"Metric: {custom_labels[i]}")
            print(f"Onboarding Test: {onboarding_test[['Test Number', 'Date']].to_dict()}")
            print(f"Most Recent Test: {most_recent_test[['Test Number', 'Date']].to_dict()}\n")

        # Extract date for the label
        latest_test_date = pd.to_datetime(most_recent_test['Date']).strftime('%m/%d/%y')
        labels.append(f"{custom_labels[i]}\n({latest_test_date})")

        # Extract percentiles and actual values
        metric_key = f"{test_type} - {sub_type} - {metric}"
        onboarding_values.append(
            percentiles[metric_key]['first_percentile'] if metric_key in percentiles else 0
        )
        recent_values.append(
            percentiles[metric_key]['recent_percentile'] if metric_key in percentiles else 0
        )
        onboarding_actuals.append(
            percentiles[metric_key]['first_value'] if metric_key in percentiles else None
        )
        recent_actuals.append(
            percentiles[metric_key]['recent_value'] if metric_key in percentiles else None
        )

    # Ensure the radar chart is circular
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    onboarding_values += onboarding_values[:1]
    recent_values += recent_values[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(14, 12), subplot_kw=dict(polar=True))

    # Plot onboarding data
    ax.plot(angles, onboarding_values, color='blue', linewidth=1, linestyle='dotted', label='Onboarding')
    ax.fill(angles, onboarding_values, color='blue', alpha=0.1)

    # Plot most recent data
    ax.plot(angles, recent_values, color='red', linewidth=1, linestyle='solid', label='Most Recent')
    ax.fill(angles, recent_values, color='red', alpha=0.1)

    # Annotate actual values with units
    for angle, onboarding_value, recent_value, metric_label in zip(angles, onboarding_actuals, recent_actuals, selected_metrics):
        unit = unit_dict.get((metric_label[0], metric_label[1]), '')

        # Determine positions based on greater and lesser value
        if onboarding_value is not None and recent_value is not None:
            if onboarding_value > recent_value:
                outer_value, inner_value = onboarding_value, recent_value
                outer_color, inner_color = 'blue', 'red'
            else:
                outer_value, inner_value = recent_value, onboarding_value
                outer_color, inner_color = 'red', 'blue'

            ax.text(angle, 95, f"{outer_value:.1f} {unit}", ha='center', fontsize=8, color=outer_color)
            ax.text(angle, 75, f"{inner_value:.1f} {unit}", ha='center', fontsize=8, color=inner_color)
        elif onboarding_value is not None:
            ax.text(angle, 80, f"{onboarding_value:.1f} {unit}", ha='center', fontsize=8, color='blue')
        elif recent_value is not None:
            ax.text(angle, 80, f"{recent_value:.1f} {unit}", ha='center', fontsize=8, color='red')

    # Finalize the radar chart
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=10)
    ax.set_title(
        f"{athlete_name} - Growth Chart\nLevel: {athlete_level}",
        fontsize=14,
        fontweight='bold'
    )
    ax.legend(loc='upper right')
    plt.tight_layout()

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)